In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
import os
import openai

openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key =openai_api_key

from langchain_openai import ChatOpenAI

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
import openai

# OpenAI API 키 설정
openai.api_key = "your-api-key"

# OpenAI 모델을 사용하여 LLMChain 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key, temperature=0.5)


# 리서치 관련 프롬프트 템플릿
research_prompt_template = """
너는 '5년간 글로벌 인구수 증가'에 관한 연구를 돕는 리서치 에이전트야. 
사용자가 요구하는 주제와 범위에 맞게 자료를 조사하고, 조사 결과를 정리하여 보고서를 제공해.

사용자 요청:
주제: {topic}
범위: {scope}
조사에 필요한 자료를 모은 후, 최종 보고서를 작성해.
"""

# PromptTemplate을 통해 템플릿에 맞는 프롬프트 설정
prompt = PromptTemplate(input_variables=["topic", "scope"], template=research_prompt_template)

# LLMChain과 프롬프트를 결합
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 리서치 관련 툴 정의
def research_tool_func(input_string):
    # LLMChain을 사용하여 리서치 수행
    return llm_chain.run(input_string)

# 툴을 정의 (위에서 작성한 함수 사용)
tools = [
    Tool(
        name="research_tool",
        func=research_tool_func,
        description="리서치 주제에 대한 질문 및 응답을 처리"
    )
]

# 에이전트를 초기화하여 멀티턴 대화 흐름 처리
agent = initialize_agent(
    tools, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm, 
    verbose=True
)

# 리서치 진행 함수
def ask_question(agent, user_input):
    response = agent.run(user_input)
    return response

# 리서치 대화 흐름
def start_research():
    print("리서치 조사 요청을 받았습니다. 5년간 글로벌 인구수 증가에 대해 차례대로 조사해보겠습니다.\n")

    # 첫 번째 질문: 주제
    user_input = input("연구 주제를 알려주세요. 예: '5년간 글로벌 인구수 증가' ")
    topic = user_input  # 사용자가 입력한 주제

    # 두 번째 질문: 조사 범위 (예: '전 세계', '대륙별', '국가별')
    user_input = input("조사 범위를 선택하세요. 예: '전 세계', '대륙별', '국가별' ")
    scope = user_input  # 사용자가 입력한 범위

    # 주제와 범위를 이용해 리서치 툴 실행
    prompt_with_values = prompt.format(topic=topic, scope=scope)  # 프롬프트에 주제와 범위 삽입
    agent_response = ask_question(agent, prompt_with_values)
    print(agent_response)

    # 세 번째 질문: 조사 기간 (예: '2018-2023', '2020-2025')
    user_input = input("조사 기간을 알려주세요. 예: '2018-2023', '2020-2025' ")
    period = user_input  # 사용자가 입력한 기간
    agent_response = ask_question(agent, f"기간: {period}")
    print(agent_response)

    # 네 번째 질문: 자료의 종류 (예: '세계은행 자료', '유엔 보고서')
    user_input = input("어떤 자료를 조사할까요? 예: '세계은행 자료', '유엔 보고서' ")
    data_type = user_input  # 사용자가 입력한 자료 종류
    agent_response = ask_question(agent, f"자료: {data_type}")
    print(agent_response)

    # 최종 보고서 제공
    print("\n리서치가 완료되었습니다! 최종 보고서를 제공합니다.")
    print(agent_response)

start_research()


리서치 조사 요청을 받았습니다. 5년간 글로벌 인구수 증가에 대해 차례대로 조사해보겠습니다.



> Entering new AgentExecutor chain...
이 주제에 대한 자료를 수집하기 위해 각 대륙의 최근 5년간 인구 증가 추세를 조사해야 합니다. 이를 위해 연구 도구를 사용하여 각 대륙별 인구 증가 데이터를 수집하겠습니다.

Action: research_tool  
Action Input: "5년간 대륙별 글로벌 인구수 증가 추세"  

ValueError: Missing some input keys: {'topic'}